# 데모 데이터 생성

In [1]:
import os
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
import json
import re
import sys
import time

In [2]:
# msn_train= pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_train.csv')
msn_dev= pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/msn_dev.csv')

In [3]:
demo_dev = pd.read_csv('/Users/baeyuna/Documents/SNU_DLab/Data/MINDdemo_dev/news.tsv', delimiter='\t', header=None)
print(demo_dev.shape)
demo_dev.head()

(18723, 8)


,0,1,2,3,4,5,6,7
0,N3112,lifestyle,lifestyleroyals,"The Brands Queen Elizabeth, Prince Charles, an...","Shop the notebooks, jackets, and more that the...",https://www.msn.com/en-us/lifestyle/lifestyler...,"[{""Label"": ""Prince Philip, Duke of Edinburgh"",...",[]
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
2,N12103,health,voices,I Was An NBA Wife. Here's How It Affected My M...,"I felt like I was a fraud, and being an NBA wi...",https://www.msn.com/en-us/health/voices/i-was-...,[],"[{""Label"": ""National Basketball Association"", ..."
3,N20460,health,medical,"How to Get Rid of Skin Tags, According to a De...","They seem harmless, but there's a very good re...",https://www.msn.com/en-us/health/medical/how-t...,"[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI...","[{""Label"": ""Skin tag"", ""Type"": ""C"", ""WikidataI..."
4,N5409,weather,weathertopstories,It's been Orlando's hottest October ever so fa...,There won't be a chill down to your bones this...,https://www.msn.com/en-us/weather/weathertopst...,"[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W...","[{""Label"": ""Orlando, Florida"", ""Type"": ""G"", ""W..."


In [4]:
demo_dev_nid = demo_dev.iloc[:,0]
print(len(demo_dev_nid))

demo_devs = msn_dev.loc[msn_dev.iloc[:,6].isin(demo_dev_nid)]
print(demo_devs.shape)
demo_devs.head()

18723
(10534, 13)


,5,nids,Title,Date,Image,Content,0,1,2,3,4,6,7
12,https://assets.msn.com/labs/mind/AABDHTv.html,AABDHTv,"50 Foods You Should Never Eat, According to He...",2019-05-21T21:10:55Z,https://img-s-msn-com.akamaized.net/tenant/amp...,What's the one food you refuse to eat? Whateve...,N22273,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,[],[]
19,https://assets.msn.com/labs/mind/AAJfTqo.html,AAJfTqo,Newark Liberty Airport's Terminal One a $2.7 b...,2019-10-23T19:44:51Z,NaN,"Gov. Phil Murphy, Senate President Stephen Swe...",N10616,travel,travelarticle,Newark Liberty Airport's Terminal One a $2.7 b...,"The project, which is the bi-state agency's si...","[{""Label"": ""Newark Liberty International Airpo...",[]
22,https://assets.msn.com/labs/mind/AAJgNxm.html,AAJgNxm,Elijah Cummings to lie in state at US Capitol ...,2019-10-24T09:23:00Z,NaN,WASHINGTON â Rep. Elijah Cummings will lie i...,N21935,news,newspolitics,Elijah Cummings to lie in state at US Capitol ...,"Cummings, a Democrat whose district included s...","[{""Label"": ""Elijah Cummings"", ""Type"": ""P"", ""Wi...","[{""Label"": ""Elijah Cummings"", ""Type"": ""P"", ""Wi..."
24,https://assets.msn.com/labs/mind/AADlunl.html,AADlunl,17 Abandoned Theme Parks to Explore for Thrill...,2019-10-02T11:00:00Z,https://img-s-msn-com.akamaized.net/tenant/amp...,Amusement parks usually conjure up images of s...,N9663,travel,traveltripideas,17 Abandoned Theme Parks to Explore for Thrill...,"Disney, Six Flags, and even the Flintstones ha...","[{""Label"": ""Amusement park"", ""Type"": ""C"", ""Wik...","[{""Label"": ""Six Flags"", ""Type"": ""O"", ""Wikidata..."
30,https://assets.msn.com/labs/mind/AAEw1Zd.html,AAEw1Zd,10 Things You Never Knew About Owls,2019-07-18T14:47:14Z,https://img-s-msn-com.akamaized.net/tenant/amp...,"There's no one ""owl""âin fact there are 250 d...",N16695,lifestyle,lifestylepetsanimals,10 Things You Never Knew About Owls,"With their huge eyes, sharp talons, and swivel...",[],[]


### 추가 수집할 데이터 

In [5]:
dev_nids = []
for nid in demo_dev.iloc[:,0].tolist():
    if nid not in demo_devs.iloc[:,6].tolist():
        dev_nids.append(nid)

print(len(dev_nids))

not_dev = demo_dev.loc[demo_dev.iloc[:,0].isin(dev_nids)]
not_dev.head() 

8189


,0,1,2,3,4,5,6,7
1,N10399,news,newsworld,The Cost of Trump's Aid Freeze in the Trenches...,Lt. Ivan Molchanets peeked over a parapet of s...,https://www.msn.com/en-us/news/world/the-cost-...,[],"[{""Label"": ""Ukraine"", ""Type"": ""G"", ""WikidataId..."
5,N21078,entertainment,gaming,Best PS5 games: top PlayStation 5 titles to lo...,Every confirmed or expected PS5 game we can't ...,https://www.msn.com/en-us/entertainment/gaming...,"[{""Label"": ""PlayStation"", ""Type"": ""J"", ""Wikida...",[]
6,N9552,health,nutrition,"50 Foods You Should Never Eat, According to He...",This is so depressing.,https://www.msn.com/en-us/health/nutrition/50-...,[],[]
7,N24180,autos,autosenthusiasts,Trying to Make a Ram 3500 as Quick as a Viper ...,The 2019 Ram 3500's new Cummins diesel has 100...,https://www.msn.com/en-us/autos/autosenthusias...,"[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat...","[{""Label"": ""Ram Pickup"", ""Type"": ""V"", ""Wikidat..."
8,N14522,health,wellness,Instagram Filters with Plastic Surgery-Inspire...,In an effort to combat some of the negative me...,https://www.msn.com/en-us/health/wellness/inst...,"[{""Label"": ""Instagram"", ""Type"": ""W"", ""Wikidata...",[]


# date 수집

### ip 우회하기

In [9]:
from selenium import webdriver

options = webdriver.ChromeOptions()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')
options.add_argument('User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36')
options.add_argument('lang=ko_KR')

PROXY = "116.254.100.165:46675" # IP:Port

webdriver.DesiredCapabilities.CHROME['proxy'] = {
    "httpProxy": PROXY,
    "ftpProxy": PROXY,
    "sslProxy": PROXY,
    "proxyType": "MANUAL"
}

chromedriver = '/Users/baeyuna/Documents/chromedriver'
driver = webdriver.Chrome(chromedriver, options=options)

In [10]:
import time
from random import *
from tqdm import notebook

MAX_SLEEP_TIME = 5
urls = not_dev.iloc[:,5].tolist()
dates = []
except_urls = []

for url in notebook.tqdm(urls[:10], desc="tqdm url going"):
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    
    try:
        driver.get(url)
        try:
            date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
            dates.append(date.get_attribute('datetime'))
        except:
            dates.append(None)
    except:
        except_urls.append(url)
    
print(len(dates))



10


In [12]:
date_df_ip = pd.DataFrame({'date':date, 'url':urls[:10]})
print('잘 뽑힌 값 개수:',date_df_ip.loc[~date_df_ip['date'].isnull()].shape)
date_df_ip

잘 뽑힌 값 개수: (10, 2)


,date,url
0,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/news/world/the-cost-...
1,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/entertainment/gaming...
2,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/health/nutrition/50-...
3,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/autos/autosenthusias...
4,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/health/wellness/inst...
5,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/travel/travelarticle...
6,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/sports/football_nfl/...
7,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/news/newspolitics/el...
8,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/lifestyle/lifestyler...
9,<selenium.webdriver.remote.webelement.WebEleme...,https://www.msn.com/en-us/health/weightloss/th...


In [ ]:
import time
from random import *
from tqdm import notebook

MAX_SLEEP_TIME = 5
urls = not_dev.iloc[:,5].tolist()
# dates = []
except_urls = []

for url in notebook.tqdm(urls[10:3000], desc="tqdm url going"):
    rand_value = randint(1, MAX_SLEEP_TIME)
    time.sleep(rand_value)
    
    try:
        driver.get(url)
        try:
            date = driver.find_element_by_xpath('//*[@id="precontent"]/section/div[1]/div/div/span[3]/time')
            dates.append(date.get_attribute('datetime'))
        except:
            dates.append(None)
    except:
        except_urls.append(url)
    
print(len(dates))

In [ ]:
date_df_ip = pd.DataFrame({'date':date, 'url':urls[:3000]})
print('잘 뽑힌 값 개수:',date_df_ip.loc[~date_df_ip['date'].isnull()].shape)
date_df_ip